In [2]:

!pip install "ibm-watsonx-ai == 0.2.6"
!pip install "langchain == 0.1.16"
!pip install "langchain-ibm == 0.1.4"
!pip install "huggingface == 0.0.1"
!pip install "huggingface-hub == 0.21.4"
!pip install "sentence-transformers == 2.5.1"
!pip install "chromadb == 0.4.24"


Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/build_env.py", line 14, in <module>
    from pip._vendor.certifi import where
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/certifi/__init__.py", line 1, in <module>
    from .core import contents, where
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/certifi/core.py", line 16, in <module>
    from importlib.resources import as_file, files
  File "/usr

In [3]:
!pip install streamlit
import streamlit as st
uploaded_file = st.file_uploader("Upload a text file", type=["txt"])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


2025-01-29 08:27:01.000 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 08:27:01.001 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 08:27:01.004 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 08:27:01.011 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 08:27:01.088 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-01-29 08:27:01.089 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [7]:
import os
import json
import gspread
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from oauth2client.service_account import ServiceAccountCredentials
from transformers import pipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from google.colab import files

def load_and_preprocess(file_path):
    """Load and split a text document into 1000-character chunks."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    with open(file_path, 'r', encoding='utf-8') as file:
        document = file.read()

    chunks = [document[i:i+1000] for i in range(0, len(document), 1000)]
    return chunks

def risk_detection(chunks):
    """Analyze legal risks using an NLP model."""
    model_name = "google/flan-t5-base"
    nlp = pipeline("text2text-generation", model=model_name)

    results = []
    for chunk in chunks:
        prompt_analysis = (
            "Analyze the following text for legal risks, hidden obligations, or compliance issues:\n\n"
            + chunk
        )
        analysis_result = nlp(prompt_analysis, max_length=200, do_sample=False)
        analysis_text = analysis_result[0]['generated_text']

        prompt_recommendations = (
            "Provide recommendations to mitigate identified risks in the following text:\n\n"
            + chunk
        )
        recommendations_result = nlp(prompt_recommendations, max_length=200, do_sample=False)
        recommendations_text = recommendations_result[0]['generated_text']

        results.append({
            "context": chunk,
            "analysis": analysis_text,
            "recommendations": recommendations_text
        })
    return results

def export_to_sheets(data, sheet_name, credentials_file):
    """Export analysis results to Google Sheets."""
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
    client = gspread.authorize(creds)

    spreadsheet = client.create(sheet_name)
    sheet = spreadsheet.sheet1

    df = pd.DataFrame(data)
    sheet.update([df.columns.values.tolist()] + df.values.tolist())

    return spreadsheet.url

def process_and_query(file_path, query, credentials_file, sheet_name):
    """Run document analysis, query Watson AI, and save results."""
    chunks = load_and_preprocess(file_path)

    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings()
    docsearch = Chroma.from_documents(texts, embeddings)

    credentials = {
        "url": "https://eu-de.ml.cloud.ibm.com",
        "apikey": "ygkx3537EypWqZ6ziVsZe_2TWa52ha7nSiCdRJAfXMBu"
    }
    project_id = "4ec8c16d-4406-4dd2-92da-41d1718164ff"

    model_id = 'google/flan-ul2'
    parameters = {
        GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
        GenParams.MIN_NEW_TOKENS: 50,
        GenParams.MAX_NEW_TOKENS: 200,
        GenParams.TEMPERATURE: 0.5
    }
    model = Model(model_id=model_id, params=parameters, credentials=credentials, project_id=project_id)
    flan_ul2_llm = WatsonxLLM(model=model)

    qa = RetrievalQA.from_chain_type(
        llm=flan_ul2_llm,
        chain_type="stuff",
        retriever=docsearch.as_retriever(),
        return_source_documents=False
    )

    result = qa.invoke(query)
    analysis = risk_detection(chunks)

    output_path = "risk_analysis.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(analysis, f, ensure_ascii=False, indent=4)

    sheet_url = export_to_sheets(analysis, sheet_name, credentials_file)
    return result, analysis, sheet_url

def send_email(email, password, recipient_email, subject, body):
    """Send an email with analysis results."""
    message = MIMEMultipart()
    message['From'] = email
    message['To'] = recipient_email
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(email, password)
            server.send_message(message)
            print("✅ Email sent successfully!")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

def main():
    print("🚀 Legal and Risk Analysis Tool in Colab 🚀")

    # Upload file
    print("📂 Upload a text file")
    uploaded_file = files.upload()
    file_path = list(uploaded_file.keys())[0]

    # User inputs
    query = input("🔍 Enter your query: ")
    credentials_file = input("📜 Enter Google Sheets credentials file path: ")
    sheet_name = input("📊 Enter Google Sheet name: ")
    email = input("✉️ Enter sender email: ")
    password = input("🔑 Enter sender email password: ")
    recipient_email = input("📩 Enter recipient email: ")

    # Run Analysis
    run_analysis = input("🛠️ Type 'yes' to analyze and send email: ")
    if run_analysis.lower() == "yes":
        result, analysis, sheet_url = process_and_query(file_path, query, credentials_file, sheet_name)

        print("\n✅ Query Result:", result)
        print("✅ Google Sheets URL:", sheet_url)
        print("✅ Analysis Results:", json.dumps(analysis, indent=4))

        subject = "Legal and Risk Analysis Results"
        body = f"Here are the results of the legal and risk analysis:\n\nQuery Result: {result}\n\nGoogle Sheets URL: {sheet_url}\n\nAnalysis Results: {json.dumps(analysis, indent=4)}"

        send_email(email, password, recipient_email, subject, body)

if __name__ == "__main__":
    main()


🚀 Legal and Risk Analysis Tool in Colab 🚀
📂 Upload a text file


Saving openlegaldata.txt to openlegaldata.txt
🔍 Enter your query: what is the final verdict?
📜 Enter Google Sheets credentials file path: alien-baton-446917-f0-a58499645f4c.json
📊 Enter Google Sheet name: google sheets
✉️ Enter sender email: chsdv2004@gmail.com
🔑 Enter sender email password: fmxe kxdz koiy qzra
📩 Enter recipient email: 218w1a0511@vrsec.ac.in
🛠️ Type 'yes' to analyze and send email: yes


Device set to use cpu



✅ Query Result: {'query': 'what is the final verdict?', 'result': "I believe that Brandconcern's complaint that the General Court awarded more in its judgment than was sought by the appellant is at least admissible. 84. (a) Inadmissibility 83. Unlike the Office, I am of the opinion that the essential information provided for in Article 21 of the Statute of the Court of Justice, read in conjunction with the first paragraph of Article 53 thereof, which is specified in Article 168 of its Rules of Procedure, is not lacking. Although those elements have not been clearly identified and no specific indication has been given as to what kind of procedural defect the judgment is alleged to contain, I believe that Brandconcern's complaint that the General Court awarded more in its judgment than was sought by the appellant is at least admissible. 84."}
✅ Google Sheets URL: https://docs.google.com/spreadsheets/d/1oDMKKgjHLW8HdCUwPEnHY_th6rncBy5HJ9cbKoxTCLs
✅ Analysis Results: [
    {
        "cont